In [80]:
import pandas as pd
import re, random
import numpy as np

In [76]:
def loadFileArr(loc):
    with open(loc) as f:
        words = f.readlines()
        words = list(map(lambda x:x[:-1], words))
    return words


wordList_orig=loadFileArr('preReq/allowed_words.txt')
# wordList_orig=loadFileArr('preReq/possible_ans.txt')

alps = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
freq = [8, 1, 2, 4, 13, 2, 2, 6, 6, 0, 0, 4, 2, 6, 7, 1, 0, 5, 6, 9, 2, 0, 2, 0, 2, 0]

freqs = pd.DataFrame({'alps': alps, 'freq': freq})
df = freqs
freqs = freqs.set_index('alps')
df['freq'] = df.freq.astype('int')
df = df.sort_values(by=['freq'], ascending=False)
df = df[df['freq']>6]
prefList = ['E','T','A','O']
df.head(10)
print(len(wordList_orig))

12972


In [77]:
setSize=5

def getMVP(arr):
    d = pd.DataFrame({'a': arr})
    d['b'] = d['a'].apply(calcVal)
    d = d.sort_values(by=['b'], ascending=False)
    return d.loc[0].a

def guess(wl):
    ls = getFilt(wl)
    hiFreq = ['a','t','o','i']
    for i in hiFreq: ls = list(filter(lambda x:x.find(i)>=0, ls))
    return ls

def eliminate(g, inWord, pred):
    ls = g
    if inWord in ls: ls.pop(ls.index(inWord))
    inarr   = list(zip([i for i in pred],[i for i in inWord]))
    remove  = list(filter(lambda x: x[0]=='0', inarr))
    
    for i in remove: ls = list(filter(lambda x:x.find(i[1])<0, ls))

    if '2' in pred:
        tmp = pred.replace('0','.')
        tmp = tmp.replace('1','.')
        look = list(filter(lambda x: x[0]=='2', inarr)) 
        look = list(map(lambda x: x[1], look))
        
        for i in look: tmp = tmp.replace('2', i,1)
        tmp = f'^{tmp}$'
        ls = list(filter(lambda x: bool(re.match(tmp, x)), ls))
        
    if '1' in pred: 
        tmp = pred.replace('0','.')
        tmp = tmp.replace('2','.')
        look = list(filter(lambda x: x[0]=='1', inarr))
        look = list(map(lambda x: x[1], look))
        for i in look: tmp = tmp.replace('1', i,1)
        tmp = f'^{tmp}$'
        ls = list(filter(lambda x: bool(re.match(tmp, x))==False, ls))
        
    return ls

def calcVal(word):
    ret = 0
    for i in word: ret+=freqs.loc[i].freq
    return ret

def validate(guess, word,g,verbose=True):
    fin=""
    val = ''
    for d,i in enumerate(guess): 
        if i in word and i==word[d]: 
            fin+='🟩'
            val+='2'
        elif i in word: 
            fin+= '🟨'
            val+='1'
        else: 
            fin+='⬛'
            val+='0'
    if verbose:
        print(fin, end = " | ")
        print(guess, end = " | ")
        print(len(g), end = " | ")
    return val
    
def getFilt(arr, drop=True):
    posibs = arr
    if drop:
        reps = list(map(lambda x: len({i for i in x}), arr))
        minSiz = max(reps)
        posibs = list(filter(lambda x:len({i for i in x})==minSiz, arr))
    return posibs
    
def select(arr, drop=True): 
    posibs = getFilt(arr, drop)
    return getMVP(posibs)

    # ser = list(zip(nu, posibs))
    # if exts: maxScore = min(nu)
    # else: maxScore = max(nu)
    # l = nu.index(maxScore)
    
    # return ser[l][1]

def run(i, srWord, verbose=True):
    turns=0
    setWord = wordList[i]
    g = wordList
    if verbose: print(f'set: {setWord}')
    pred = validate(srWord, setWord, g,verbose)

    turns+=1
    if verbose: print(turns)
    dro = True
    while pred!= '22222' and turns!=5:
        turns+=1
        g = eliminate(g, srWord, pred)
        # if turns==3: dro, ext = False, False
        srWord = select(g, dro)
        pred = validate(srWord,setWord, g,verbose)
        if verbose: print(turns)
    return sum([int(i) for i in pred])

def avg(ret, div): return (sum(ret)/div)*100


guessArr = guess(wordList_orig)
# for i in list(range(10)):
#     r = random.randint(0,2314)
# #     run(r, 'ratio')
#     run(r, guessArr[0])

In [78]:
# bench mark
su = []
for i in list(range(100)):
        r = random.randint(0,2314)
        su.append(run(r,'patio', False))
        
set(su)
len(list(filter(lambda x: x==10, su)))

70

In [79]:
for i in list(range(10)):
    r = random.randint(0,2314)
    run(r, 'atone')

set: udder
⬛⬛⬛⬛🟨 | atone | 2313 | 1
⬛🟨⬛⬛⬛ | belch | 361 | 2
🟨⬛⬛🟩🟩 | diver | 88 | 3
⬛🟨⬛🟩🟩 | super | 8 | 4
🟩🟩🟩🟩🟩 | udder | 2 | 5
set: blunt
⬛🟨⬛🟩⬛ | atone | 2312 | 1
🟩🟩⬛🟩⬛ | blind | 51 | 2
🟩🟩🟩🟩🟩 | blunt | 1 | 3
set: click
⬛⬛⬛⬛⬛ | atone | 2312 | 1
⬛🟨⬛🟩⬛ | birch | 191 | 2
⬛🟩🟩🟩🟩 | flick | 8 | 3
⬛🟩🟩🟩🟩 | slick | 2 | 4
🟩🟩🟩🟩🟩 | click | 1 | 5
set: juror
⬛⬛🟨⬛⬛ | atone | 2312 | 1
⬛⬛🟩⬛⬛ | birch | 269 | 2
⬛🟨🟩🟨⬛ | forum | 13 | 3
⬛🟩🟩⬛⬛ | surly | 8 | 4
🟩🟩🟩🟩🟩 | juror | 1 | 5
set: medal
🟨⬛⬛⬛🟨 | atone | 2312 | 1
🟨⬛🟨🟨🟨 | abled | 844 | 2
🟨⬛⬛⬛🟨 | acrid | 725 | 3
🟨🟨⬛🟨⬛ | amply | 206 | 4
🟨⬛⬛🟨⬛ | askew | 102 | 5
set: fishy
⬛⬛⬛⬛⬛ | atone | 2312 | 1
⬛🟩⬛⬛🟨 | birch | 191 | 2
⬛🟩⬛⬛🟩 | dimly | 25 | 3
🟩🟩🟩🟩🟩 | fishy | 8 | 4
set: tasty
🟨🟨⬛⬛⬛ | atone | 2312 | 1
🟨⬛⬛⬛⬛ | acrid | 584 | 2
⬛🟩⬛⬛🟩 | balmy | 126 | 3
⬛🟩⬛⬛🟩 | gawky | 15 | 4
⬛🟩🟩🟩🟩 | hasty | 13 | 5
set: ovine
⬛⬛🟨🟩🟩 | atone | 2312 | 1
⬛🟨⬛🟩🟩 | borne | 12 | 2
🟩⬛🟩🟩🟩 | opine | 8 | 3
🟩🟩🟩🟩🟩 | ovine | 1 | 4
set: acute
🟩🟨⬛⬛🟩 | atone | 2312 | 1
🟩⬛⬛⬛🟩 | abide | 26 | 2
🟩🟩🟩🟩🟩 | ac

In [51]:
wordList.index('overt')

1355

In [101]:
a = ['a','b']
# a.pop(a.index('a'))
# print(a)
a.index('c')

ValueError: 'c' is not in list

In [86]:
a = np.array([1,1,0,0,2]).T
b = np.array([1,1,0,0,2])
np.dot(a,b)

6